# 01.05 rootfinding: without derivatives

##### code, support for section text

In [ ]:
if False: # settings for sensei
  from google.colab import auth
  from googleapiclient.discovery import build

  auth.authenticate_user()
  about = build('drive','v3').about().get(fields='user').execute()
  if about['user']['displayName'] == "Sun K.":

    # output to pdf setting
    from google.colab import output
    output.no_vertical_scroll()
    #output.no_horizontal_scroll() # sigh, doesnt exist

In [ ]:
if True: # settings for the people
  import cmath as cm
  from copy import deepcopy # lists, numpy have their own but bc scipy # vanilla secant
  from itertools import count
  import math as m
  import matplotlib.animation
  import matplotlib.pyplot as plt
  import numpy as np
  import numpy.polynomial as npp
  import scipy as sp
  import statistics as st
  import textwrap
  from tabulate import tabulate


###### code, secant

In [ ]:
# algorithm, expanded for lecture 01_02 # mod for secant

def fpi_secant(g,x,tol=1e-8,max_iter=100,worksheet=False):

  count = 0
  if worksheet:
    data = []

  gx = g(x)
  if worksheet:
    data.append([count,x[0]])
    count += 1
    data.append([count,x[1]])
  while (abs(gx[1]-x[1]) > tol) and (count < max_iter):
    x = gx
    gx = g(x)

    count += 1
    if worksheet:
      data.append([count,x[1]])
  if worksheet:
    return data
  else:
    return x


###### code, regular falsi

In [ ]:
# algorithm, basic

def secant_rf(f,ab,tol=1e-8):

  a = ab[0]; b = ab[1]
  fa = f(a); fb = f(b)

  while abs(b-a) > tol:
    c = (b*fa - a*fb)/(fa-fb)
    fc = f(c)
    if fc == 0:
      break
    else:
      if fa*fc < 0:
        b = c
        fb = fc
      else:
        a = c
        fa = fc

  return c


In [ ]:
# algorithm, basic # modified bisect_expanded from lecture 01_01

def secant_rf_expanded(f,ab,tol=1e-8,all=False,workspace=False):

  if all:
    iterations = 0
  if workspace:
    ws = []
    i = 0

  a = ab[0]; b = ab[1]
  fa = f(a); fb = f(b) # <-- ADDED

  while abs(b-a) > tol:
    c = (b*fa - a*fb)/(fa-fb) # <-- MODDED
    fc = f(c)
    fs = 1 if fc > 0 else -1 if fc < 0 else 0 # bc all one dtype

    if fc == 0:
      if workspace:
        ws.append([i,a,fa,b,fb,c,fc,fs])
      break;
    if not 'fa' in locals(): # ie, local to function
      fa = f(a)
      if workspace:
        fb = f(b)
    if workspace:
      ws.append([i,a,fa,b,fb,c,fc,fs])
      i += 1

    if fa*fc < 0:
      b = c
      if workspace:
        fb = fc
    else:
      a = c
      fa = fc

    if all:
      iterations += 1
  if all:
    if workspace:
      return c,(a,b),iterations,ws
    else:
      return c,(a,b),iterations
  else:
    if workspace:
      return c,ws
    else:
      return c


###### code, mullers

In [ ]:
# https://en.wikipedia.org/wiki/Muller%27s_method # mod

# newtons divided difference
def dd(f,xx):
  if len(xx) == 2:
    a,b = xx
    return (f(b)-f(a))/(b-a)
  return (dd(f,xx[1:]) - dd(f,xx[0:-1])) / (xx[-1] - xx[0])

def mullers_mod(f,xs,max_iter=100,tol=1e-8,method=None):
  """
  method: 0 complex, 1 real, 2 real
  """
  rc = []

  i = 0
  x0,x1,x2 = xs
  e = x2 - x1
  rc.append([i,x0,x1,x2,e])

  while (abs(e)>tol) and (i<max_iter):
    w = dd(f,(x2,x1)) + dd(f,(x2,x0)) - dd(f,(x2,x1))
    if method == 1: # 86 complex before sqrt
      sd = np.sqrt(abs(pow(w,2) - 4*f(x2)*dd(f,(x2,x1,x0)))) # LOL!!
    else:
      sd = cm.sqrt(pow(w,2) - 4*f(x2)*dd(f,(x2,x1,x0)))
    ds = [w+sd,w-sd]
    x3 = x2 - 2*f(x2) / max(ds,key=abs) # max denominator

    # next
    i += 1
    if method == 2: # 86 complex after sqrt
      x0,x1,x2 = x1,x2,x3.real
    else:
      x0,x1,x2 = x1,x2,x3
    e = x2 - x1
    rc.append([i,x0,x1,x2,e])

  return np.array(rc)


In [ ]:
# instead of complex number hack, use future methods

def mullers_lol(f,xs,max_iter=100,tol=1e-8,method=0,show=True):
  """
  method: displays roots but really placeholder to overload mullers_mod
  """
  rc = []

  i = 0
  x0,x1,x2 = xs
  e = x2 - x1
  rc.append([i,x0,x1,x2,e])

  while (abs(e)>tol) and (i<max_iter):
    lol = np.array([x0,x1,x2])
    pc = np.polyfit(lol,f(lol),deg=2)
    root1,root2 = np.roots(pc)
    if method > 0 and show==True:
      print(f"step {i}, parabola: {np.polynomial.Polynomial(np.flip(pc))}")
      print(f"           roots: [{root1},{root2}]\n\n")
    x3 = root1 if abs(x2 - root1) < abs(x2 - root2) else root2
    #if abs(x2 - root1) < abs(x2 - root2):
    #  x3,na = root1,root2
    #else:
    #  x3,na = root2,root1

    # next
    i += 1
    x0,x1,x2 = x1,x2,x3
    e = x2 - x1
    rc.append([i,x0,x1,x2,e])

  return np.array(rc)


###### code, iqi

In [ ]:
def iqi(f,xs,max_iter=100,tol=1e-8,doyourworst=False,workspace=False):

  if workspace:
    ws = []

  i = 0
  abc = np.array(xs) #,dtype=float)
  a,b,c = abc
  d = c
  e = [abs(xi - d) for xi in [a,b]] # fwe
  emin = min(e)
  if workspace:
    ws.append([i,a,b,d,emin])

  if emin > tol:
    while i < max_iter:
      fa = f(a); fb = f(b); fc = f(c)
      q = fa/fb; r = fc/fb; s = fc/fa
      d = c - (r*(r-q)*(c-b) + (1-r)*s*(c-a)) / ((q-1)*(r-1)*(s-1))
      fd = f(d)

      e = [abs(fx - fd) for fx in [fa,fb,fc]] # bwe here is Δy bc wrt root
      emin = min(e)
      if emin > tol:
        if doyourworst:
          ie = e.index(max(e))
          abc[ie] = d
        else:
          abc = np.array([b,c,d])
      else:
        if workspace:
          ie = e.index(max(e))
          abc[ie] = d
          ws.append([i,abc[0],abc[1],abc[2],emin])
          break
        else:
          return d

      i += 1
      a,b,c = abc
      if workspace:
        ws.append([i,a,b,c,emin])

  elif doyourworst and not workspace:
    ie = 2

  if workspace:
    return np.array(ws)
  else:
    if doyourworst:
      return abc[ie]
    else:
      return abc[2]


###### code, brent

In [ ]:
# https://blogs.mathworks.com/cleve/2015/10/12/zeroin-part-1-dekkers-algorithm/

# without iqi

def zeroin(f,a,b,display=0):

  fa = f(a)
  fb = f(b)
  if fa*fb > 0:
    raise Exception("interval does not bracket root.")

  if display > 0:
    print(f"{1:5.0f} initial {a:19.15f} {fa:23.15e}")
    print(f"{2:5.0f} initial {b:19.15f} {fb:23.15e}")
  k = 2
  c = a; fc = fa

  while True:
    # reorder
    if fb*fc > 0:
      c = a
      fc = f(a)
    if fc < fb:
      #x = a; fx = fa
      a = b; fa = fb
      b = c; fb = fc
      c = a; fc = fa

    # midpoint
    m = (b+c)/2
    if abs(m-b) <= np.spacing(abs(b)): # so many iterations with this logical
      return m,k

    # try secant, Δ = p/q
    p = (b-a)*fb
    if p >= 0:
      q = fa - fb
    else:
      q = fb - fa
      p *= -1

    # save point
    a = b; fa = fb
    k += 1

    if display>0 and k>display:
      print(f"{k:5.0f} *** {p:19.15e} vs {q:19.15e}")
      print(f"{k:5.0f} *** {p <= np.spacing(q)} <= {np.spacing(q):19.15e}")
      print(f"{k:5.0f} *** {p <= (m-b)*q} <= {(m-b)*q:19.15e}")

    # next point
    if p <= np.spacing(q):
      b = b + np.spacing(b)*(-1 if c < b else 1)
      fb = f(b)
      s_method = "minimal"

    elif p <= (m-b)*q:
      b += p/q
      fb = f(b)
      s_method = "secant "

    else:
      b = m
      s_method = "bisect "
    fb = f(b)
    if display>0:
      if k>display:
        print(f"{k:5.0f} *** {a:19.15e} {b:19.15e} {c:19.15e}")
      print(f"{k:5.0f} {s_method} {b:19.15f} {fb:23.15e}")


###### code, examples

In [ ]:
# example 16, secant method for example 01

def eg_16(show="ani"):
  """
  show : "ani" animation, "data (as table)
  """

  # prob-def
  f = lambda x: pow(x,3) + x - 1
  df = lambda x: 2*pow(x,2) + 1
  df2 = lambda x: 4*x
  dq = lambda x1,x0 : (f(x1)-f(x0))/(x1-x0) # kinda lotta same calls = meh?
  def g(x): # secant method
    rc = deepcopy(x) # not necessarily numpy
    rc[1] = x[1] - f(x[1])/dq(x[1],x[0])
    rc[0] = x[1]
    return rc

  ab = (-1,1)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = f"example 16 ~ example 01 w secants"
  s_label = "{x^3 + x - 1}{3x^2 + 1}"
  s_label = f"$g(x) = x - \\frac{s_label}$"

  # prob-def, runtime
  x0 = [0,1]
  # calc
  root = (sp.optimize.root(f,[x0[0]])).x
  ws = fpi_secant(g,x0,tol=5e-16,worksheet=True)

  # cfg, runtime
  iterations = 4

  # exxing types
  α = (1 + m.sqrt(5))/2 # for supralinear convergence
  sm = pow(abs(df2(root)/(2*df(root))),α-1)
  s_title += f"\nand expected rate of convergence: {sm[0]}"
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/pow(e0,α)
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  if show == "ani": # output, animiation # True requires "ani" in a separate code cell

    # data, true, scatter
    h = 0.1
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    h_plot = h/10
    xs_plot_a = ab[0]-h
    xs_plot_b = ab[1]+h
    xs_plot = np.arange(xs_plot_a,xs_plot_b+h_plot,h_plot)
    ys_plot = f(xs_plot)

    # plot, cfg
    plt.close("all")
    fig,ax = plt.subplots()
    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    iani = count() # used in animate()

    # plot, actual
    ax.plot(xs_plot,ys_plot,zorder=1)
    ax.set_xlim(xs_plot_a,xs_plot_b)
    ymin = m.floor(ys_plot.min()/h)*h - h
    ymax = m.ceil(ys_plot.max()/h)*h + h
    ax.set_ylim(ymin,ymax)
    ax2 = ax.twinx()
    ax.set_title(s_title,pad=10)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    def animate(t):
      ax2.cla()
      k = next(iani)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=200,zorder=2)
      if k > 2:
        if k < 5:
          step = 2
          di = 0
        else:
          step = 3
          di = 1
        imax = k - 2 + di # first frame on ax2 # x0, no secant
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):
            if j > 1:
              if i > 0:
                x = ws[i,1]
                x1 = ws[i+1,1]
                y = f(x)
                ya = y*((xs_plot_a-x)/(x-x1) + 1)
                yb = y*((xs_plot_b-x)/(x-x1) + 1)
                ax2.plot([xs_plot_a,xs_plot_b],[ya,yb],label="$m_{i}$",color=color)
              else:
                j += 1
            else:
              if j == 0:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [0]
                acbs = ["x"]
              elif j == 1:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [f(x)]
                acbs = ["y"]
              ax2.scatter(xs_j,ys_j,color=color,zorder=zorder+i)
              for acb,x,y in zip(acbs,xs_j,ys_j):
                label = "$" + acb+ "_{" + str(i) + "}$"
                ax2.text(x-h_plot*2,y-h*2,label,color=color,fontweight="bold",zorder=zorder*2)

      # inside ani function!!
      ax2.set_xlim(xs_plot_a,xs_plot_b)
      ax2.set_ylim(ymin,ymax)

    n_frames = 2 + 2 + (iterations-1)*3 + 1 # root + x0 + (i-1)*(x+y+m) + 1
    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=n_frames,interval=n_frames)
    return ani

  else: # output, table
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".14e"
    else:
      s_ffmt = ".14f"
    print(f"\n{s_title}. x0 = {x0}.\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]^α'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]^α'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))


In [ ]:
# example 17, regula falsi

def eg_17(show="ani"):
  """
  show : "ani" animation, "data (as table)
  """

  # problem
  f = lambda x: pow(x,3) - 2*pow(x,2) + 1.5*x
  ab = (-1.,1.)
  tol = 1e-08

  # prob-def, generalized
  s_title = f"regula falsi: $x^3 + 2x^2 - 1.5x$, $x \in[{ab[0]},{ab[1]}]$\n"

  # prob-def, runtime
  # calc
  root,ws = secant_rf_expanded(f,ab,tol,workspace=True)

  # cfg, runtime
  iterations = 4

  if show == "ani": # output, plot # True requires "ani" in a separate code cell
    # data, true, scatter
    h = 0.1
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    h_plot = h/10
    xs_plot = np.arange(ab[0]-h,ab[1]+h+h_plot,h_plot)
    ys_plot = f(xs_plot)

    # plot, cfg
    plt.close("all")
    fig,ax = plt.subplots()
    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    iani = count() # used in animate()

    # plot, actual
    dsize = 200 # lol if you try to show 5+ iterations BC WHY WOULD YOU EVER?!
    size = dsize*iterations + dsize/2
    ax.plot(xs_plot,ys_plot,zorder=1)
    #ax.scatter(xs,ys,zorder=3) #equidistant actual unneeded

    # plot, style
    xmin = xs_plot.min()
    xmax = xs_plot.max()
    ax.set_xlim(xmin,xmax)
    ymin = m.floor(ys_plot.min()/h)*h - h
    ymax = m.ceil(ys_plot.max()/h)*h + h
    ax.set_ylim(ymin,ymax)
    ax2 = ax.twinx()
    ax.set_title(s_title)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    step = 2
    def animate(t):
      ax2.cla()
      offsets = {}
      k = next(iani)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=size,zorder=2)
      if k > 2:
        imax = k - 2 # first frame on ax2
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):
            if j == 0:
              xs_j = [ws[i,1],ws[i,3]]
              ys_j = [ws[i,2],ws[i,4]]
              acbs = ["a","b"]
              ax2.plot(xs_j,ys_j,color=color)
            else:
              xs_j = [ws[i,5]]
              ys_j = [ws[i,6]]
              acbs = ["c"]
            ax2.scatter(xs_j,ys_j,color=color,marker=".",s=size-i*dsize,zorder=zorder+i)
            for acb,x,y in zip(acbs,xs_j,ys_j):
              label = "$" + acb+ "_{" + str(i) + "}$"
              if x in offsets:
                offsets[x] += 1
              else:
                offsets[x] = 1
              ax2.text(x-h_plot*2,y-h*2-h*3*offsets[x],label,color=color,fontweight="bold",zorder=zorder*2)

      ax2.set_xlim(xmin,xmax)
      ax2.set_ylim(ymin,ymax)

    n_frames = 2 + iterations*step
    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=n_frames,interval=n_frames)
    return ani

  else: # output, table
    from tabulate import tabulate
    print(s_title)
    if True: # use iterations
      print(tabulate(ws[0:iterations][:],headers=["i","a","f(a)","b","f(b)","c","f(c)","±"],intfmt="03d",floatfmt=".8f",tablefmt="github"))
    else:
      print(tabulate(ws,headers=["i","a","f(a)","b","f(b)","c","f(c)","±"],intfmt="03d",floatfmt=".8f",tablefmt="github"))


In [ ]:
# example 17, secant method

def eg_17a(show="ani"):
  """
  show : "ani" animation, "data" (as table)
  """

  # problem
  f = lambda x: pow(x,3) - 2*pow(x,2) + 1.5*x
  df = lambda x: 3*pow(x,2) - 4*x + 1.5
  df2 = lambda x: 6*x - 4
  dq = lambda x1,x0 : (f(x1)-f(x0))/(x1-x0) # kinda lotta same calls = meh?
  def g(x): # secant method
    rc = deepcopy(x) # not necessarily numpy
    rc[1] = x[1] - f(x[1])/dq(x[1],x[0])
    rc[0] = x[1]
    return rc

  # prob-def, interval
  ab = (-1,1)
  tol = 1e-08
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = f"secant: $x^3 + 2x^2 - 1.5x$, $x \in[{ab[0]},{ab[1]}]$\n"

  # prob-def, runtime
  x0 = [ab[0],ab[1]]
  # calc
  root = (sp.optimize.root(f,[x0[0]])).x
  ws = fpi_secant(g,x0,tol=tol,worksheet=True)

  # cfg, runtime
  iterations = 4

  # exxing types
  α = (1 + m.sqrt(5))/2 # for supralinear convergence
  sm = pow(abs(df2(root)/(2*df(root))),α-1)
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/pow(e0,α)
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  if show == "ani": # output, plot # True requires "ani" in a separate code cell
    # data, true, scatter
    h = 0.1
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    h_plot = h/10
    xs_plot_a = ab[0]-h
    xs_plot_b = ab[1]+h
    xs_plot = np.arange(xs_plot_a,xs_plot_b+h_plot,h_plot)
    ys_plot = f(xs_plot)

    # plot, cfg
    plt.close("all")
    fig,ax = plt.subplots()
    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    iani = count() # used in animate()

    # plot, actual
    dsize = 200 # lol if you try to show 5+ iterations BC WHY WOULD YOU EVER?!
    size = dsize*iterations + dsize/2
    ax.plot(xs_plot,ys_plot,zorder=1)
    #ax.scatter(xs,ys,zorder=3) #equidistant actual unneeded

    # plot, style
    ax.set_xlim(xs_plot_a,xs_plot_b)
    ymin = m.floor(ys_plot.min()/h)*h - h
    ymax = m.ceil(ys_plot.max()/h)*h + h
    ax.set_ylim(ymin,ymax)
    ax2 = ax.twinx()
    ax.set_title(s_title)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    step = 2
    def animate(t):
      ax2.cla()
      offsets = {}
      k = next(iani)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=200,zorder=2)
      if k > 2:
        if k < 5:
          step = 2
          di = 0
        else:
          step = 3
          di = 1
        imax = k - 2 + di # first frame on ax2 # x0, no secant
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):
            if j > 1:
              if i > 0:
                x = ws[i,1]
                x1 = ws[i+1,1]
                y = f(x)
                ya = y*((xs_plot_a-x)/(x-x1) + 1)
                yb = y*((xs_plot_b-x)/(x-x1) + 1)
                ax2.plot([xs_plot_a,xs_plot_b],[ya,yb],label="$m_{i}$",color=color)
              else:
                j += 1
            else:
              if j == 0:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [0]
                acbs = ["x"]
              elif j == 1:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [f(x)]
                acbs = ["y"]
              ax2.scatter(xs_j,ys_j,color=color,zorder=zorder+i)
              for acb,x,y in zip(acbs,xs_j,ys_j):
                label = "$" + acb+ "_{" + str(i) + "}$"
                ax2.text(x-h_plot*2,y-h*2,label,color=color,fontweight="bold",zorder=zorder*2)

      # inside ani function!!
      ax2.set_xlim(xs_plot_a,xs_plot_b)
      ax2.set_ylim(ymin,ymax)

    n_frames = 2 + iterations*step
    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=n_frames,interval=n_frames)
    return ani

  else: # output, table
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".14e"
    else:
      s_ffmt = ".14f"
    print(s_title)
    print(f"expected rate of convergence: {sm}\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]^α'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]^α'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))


In [ ]:
# example 17, mullers method w modified example

def eg_17c(show="ani"):

  # prob-def
  #f = lambda x: pow(x,3) + x - 1 # ab = (0,1) # example 16 ~ example 01
  #f = lambda x: pow(x,3) - 2*pow(x,2) + 1.5*x # ab = (-1,1)
  f = lambda x: pow(x,3) - pow(x,2) - x - 1 # ab = (-2,2) # r = 1.83

  # prod-def, interval
  ab = (-2,2)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  #mullers = mullers_mod
  mullers = mullers_lol
  s_title = f"modified example 17 w mullers"
  #s_label = "$x^3 + x - 1$" # example 16 ~ example 01
  #s_label = "$x^3 - 2x^2 - 1.5x$"
  s_label = "$x^3 - x^2 - x - 1$"

  # prob-def, runtime
  # calc
  showEqns = False if show=="ani" else True
  ws = mullers(f,xs=[ab[0],st.mean(ab),ab[1]],max_iter=5,method=2,show=showEqns) # meh on complex!!

  # cfg, runtime (animation)
  iterations = 3

  # exxing types
  #root = (sp.optimize.root(f,st.mean(ab))).x
  root = (sp.optimize.root(f,1.9)).x
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[3]-root)
    if e0 != 0:
      ed = ei/e0
    else:
      ed = None
    e0 = ei
    we.append([int(w[0]),w[1],w[2],w[3],w[4],ei,ed])

  if show=="ani": # output, plot # True requires "ani" in a separate code cell

    # data, true, scatter
    h = (ab[1]-ab[0])/10
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    dx = h/10 # scaled plot offset
    xs_plot = np.arange(ab[0]-h,ab[1]+h+dx,dx)
    ys_plot = f(xs_plot)

    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    fig,ax = plt.subplots()
    iani = count() # used in animate()

    # plot, actual
    dsize = 200 # lol if you try to show 5+ iterations BC WHY WOULD YOU EVER?!
    size = dsize*iterations + dsize/2
    ax.plot(xs_plot,ys_plot,zorder=1)
    #ax.scatter(xs,ys,zorder=3) # equidistant unneeded
    xmin = xs_plot.min()
    xmax = xs_plot.max()
    ax.set_xlim(xmin,xmax)
    ymin = m.floor(ys_plot.min()/h)*h - h
    ymax = m.ceil(ys_plot.max()/h)*h + h
    dy = (ymax-ymin)/10/10 # scaled plot offset
    ax.set_ylim(ymin,ymax)
    ax2 = ax.twinx()
    ax.set_title(s_title)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    step = 3
    def animate(t):
      ax2.cla()
      offsets = {}
      k = next(iani)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=size,zorder=2)
      if k > 2:
        imax = k - 2 # first frame on ax2
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):

            if j == 1: # parabola
              # same xs_j,ys_j as j == 0
              pc = np.polyfit(xs_j,ys_j,deg=2)
              ps_plot = np.polyval(pc,xs_plot)
              ax2.plot(xs_plot,ps_plot,color=color)

            else:
              if j == 0: # 3 pts for parabola
                xs_j = np.array([ws[i,1],ws[i,2],ws[i,3]])
                ys_j = f(xs_j)
                acbs = ["a","b","c"]
                s_marker = "."

              else: # new point at parabola intersection
                xs_j = np.array([ws[i+1,3]])
                ys_j = f(xs_j)
                acbs = ["d"]
                s_marker = "*"
              ax2.scatter(xs_j,ys_j,color=color,marker=s_marker,s=size-i*dsize,zorder=zorder+i)
              for acb,x,y in zip(acbs,xs_j,ys_j):
                label = "$" + acb+ "_{" + str(i) + "}$"
                if x in offsets:
                  offsets[x] += 1
                else:
                  offsets[x] = 1
                ax2.text(x-dx*2,y-dy*6-dy*4*(offsets[x]-1),label,color=color,fontweight="bold",zorder=zorder*2)

      ax2.set_xlim(xmin,xmax)
      ax2.set_ylim(ymin,ymax)

    n_frames = 2 + iterations*step
    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=n_frames,interval=n_frames)
    return ani

  else: # output, table
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}: {s_label}\n")
    s_headers = ["i","a","b","c","gap","e[i]",'"/e[i-1]']
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=s_headers,intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=s_headers,intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))


In [ ]:
# example 17, iqi w modified example

def eg_17d(show="ani"):

  # prob-def
  #f = lambda x: pow(x,3) + x - 1 # ab = (0,1) # example 16 ~ example 01
  #f = lambda x: pow(x,3) - 2*pow(x,2) + 1.5*x # ab = (-1,1)
  f = lambda x: pow(x,3) - pow(x,2) - x - 1.

  # prod-def, interval
  ab = (1.,2.)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = f"modified example 17 w iqi"
  #s_label = "$x^3 + x - 1$" # example 16 ~ example 01
  #s_label = "$x^3 - 2x^2 - 1.5x$"
  s_label = "$x^3 - x^2 - x - 1$"

  # prob-def, runtime
  # calc
  doyourworst = True
  ws = iqi(f,xs=[ab[0],st.mean(ab),ab[1]],doyourworst=doyourworst,workspace=True)

  # cfg, runtime (animation)
  iterations = 3

  # exxing types
  root = (sp.optimize.root(f,x0=1.83)).x # lolwut
  we = []; e0 = 0
  for i,w in enumerate(ws):
    if doyourworst:
      if i == 0:
        di = 1
      else:
        di = -1
      for ix in range(1,len(w)): # 1,2,3 # 4 is bwe
        if w[ix] != ws[i+di][ix]:
          xi = w[ix]
          break
    else:
      xi = w[3]
    ei = abs(xi-root)
    if e0 != 0:
      ed = ei/e0
    else:
      ed = None
    e0 = ei
    we.append([i,w[1],w[2],w[3],w[4],ei,ed])

  # output, animation # True requires "ani" in a separate code cell
  if show == "ani":

    # data, true, scatter
    h = (ab[1]-ab[0])/10
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    dx = h/10 # scaled plot offset
    xs_plot = np.arange(ab[0]-h,ab[1]+h+dx,dx)
    ys_plot = f(xs_plot)

    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    fig,ax = plt.subplots()
    iani = count() # used in animate()

    # plot, actual
    dsize = 200 # lol if you try to show 5+ iterations BC WHY WOULD YOU EVER?!
    size = dsize*iterations + dsize/2
    ax.plot(xs_plot,ys_plot,zorder=1)
    #ax.scatter(xs,ys,zorder=3) # equidistant unneeded
    xmin = xs_plot.min()
    xmax = xs_plot.max()
    ax.set_xlim(xmin,xmax)
    ymin = m.floor(ys_plot.min()/h)*h - h
    ymax = m.ceil(ys_plot.max()/h)*h + h
    dy = (ymax-ymin)/10/10 # scaled plot offset
    ax.set_ylim(ymin,ymax)
    ax2 = ax.twinx()
    ax.set_title(s_title)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    step = 3
    def animate(t):
      ax2.cla()
      offsets = {}
      k = next(iani)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=size,zorder=2)
      if k > 2:
        imax = k - 2 # first frame on ax2
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):

            if j == 1: # parabola
              # same xs_j,ys_j as j == 0
              pc = np.polyfit(ys_j,xs_j,deg=2)
              ps_plot = np.polyval(pc,ys_plot)
              ax2.plot(ps_plot,ys_plot,color=color)

            else:
              if j == 0: # 3 pts for parabola
                xs_j = np.array([ws[i,1],ws[i,2],ws[i,3]])
                ys_j = f(xs_j)
                acbs = ["a","b","c"]
                s_marker = "."

              else: # new point at parabola intersection
                if doyourworst:
                  for iw in range(1,4): # 1,2,3
                    if ws[i+1,iw] != ws[i,iw]:
                      break
                else:
                  iw = 3
                xs_j = np.array([ws[i+1,iw]])
                ys_j = f(xs_j)
                acbs = ["d"]
                s_marker = "+"
              ax2.scatter(xs_j,ys_j,color=color,marker=s_marker,s=size-i*dsize,zorder=zorder+i)
              for acb,x,y in zip(acbs,xs_j,ys_j):
                label = "$" + acb+ "_{" + str(i) + "}$"
                if x in offsets:
                  offsets[x] += 1
                else:
                  offsets[x] = 1
                ax2.text(x-dx*2,y-dy*6-dy*4*(offsets[x]-1),label,color=color,fontweight="bold",zorder=zorder*2)

      ax2.set_xlim(xmin,xmax)
      ax2.set_ylim(ymin,ymax)

    n_frames = 2 + iterations*step
    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=n_frames,interval=n_frames)
    return ani

  else: # output, table
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}: {s_label} and root {root}\n")
    s_headers = ["i","a","b","c","bwe (Δy)","e[i]",'"/e[i-1]']
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=s_headers,intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=s_headers,intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))


## 0 intro

what if $f(x)$ has no (or unknown) $f'(x)$?

## 1 secant method, variants

### secant method

replace the derivative with a difference quotient. ie, replace tangent line with secant line through previous two guesses. ie, approximation for derivative at $x_i$ is difference quotient

$$q(x_i,x_{i-1}) = \frac{f(x_i)-f(x_{i-1})}{x_i-x_{i-1}}.$$

##### algorithm <b>secant method</b>

\begin{align}
  x_0,x_1 &= \text{initial guesses} \\
  \\
  x_{i+1} &= x_i - f(x_i)\cdot \underbrace{\frac{x_i-x_{i-1}}{f(x_i)-f(x_{i-1})}}_{\sim \tfrac{1}{f'(x_i)}}, \quad i = 1,2,3,\dots \\ \\
  x_{i+1} &= x_i - \frac{f(x_i)}{q(x_i,x_{i-1})}, \quad i = 1,2,3,\dots
\end{align}

```
icount = 0

fx_old = f(x_old)
if abs(fx_old) < epsilon # epsilon ~ eta
  return x_old
fx_older = f(x_older)
if abs(fx_older) < epsilon # epsilon ~ eta
  return x_older

dq = (fx_old - fx_older)/(x_old - x_older)
x_new = x_old - fx_old/dq
fx = f(x_new)
icount = icount + 1

# while (abs(fx) > epsilon) and (icount <= imax): # epsilon ~ eta
while (abs(x_new - x_old) > epsilon) and (icount <= imax):
  x_older = x_old
  fx_older = fx_old
  x_old = x_new
  fx_old = fx
  dq = (fx_old - fx_older)/(x_old - x_older)
  x_new = x_old - fx_old/dq
  fx = f(x_new)
  icount = icount + 1

return x_new
```

#### convergence

assume that method converges to $r$ and $f'(r) \ne 0$, then the approximate error relationship

$$e_{i+1} \approx \left| \frac{f''(r)}{2f'(r)} \right|\, e_ie_{i-1}$$

holds and implies

$$e_{i+1} \approx \left| \frac{f''(r)}{2f'(r)} \right|^{\alpha -1} e_i^{\alpha}, \quad$$

where $\alpha = \frac{1 + \sqrt{5}}{2} \approx 1.62$. secant method convergence to simple roots is called <b>superlinear</b>, meaning that it lies between linearly and quadratically convergent methods.

##### example 16

example 01, revisted. apply secant method with $x_0 = 0, x_1 = 1$ to find root of $f(x) = x^3 + x - 1$.

\begin{align}
  x_{i+1} &= x_i = \frac{(x_i^3 + x_i - 1)(x_i - x_{i-1})}{x_i^3 + x_i - (x_{i-1}^3 + x_{i-1})} \\
  \\
  &\Downarrow \quad x_0 = 0, x_1 = 1 \\
  \\
  x_2 &= 1 - \frac{(1)(1-0)}{(1+1-0} = \frac{1}{2} \\
  x_3 &= \frac{1}{2} - \frac{-\tfrac{3}{8}(\tfrac{1}{2}-1)}{\tfrac{3}{8} - 1} = \frac{7}{11}.
\end{align}

In [ ]:
ani = eg_16()
ani

In [ ]:
eg_16(show="data")


example 16 ~ example 01 w secants
and expected rate of convergence: 0.8068742995873851. x0 = [0, 1].

|   i |             x[i] |             e[i] |               "/e[i-1]^α |
|-----|------------------|------------------|--------------------------|
| 000 | 0.00000000000000 | 0.68232780382802 |                          |
| 001 | 1.00000000000000 | 0.31767219617198 |         0.58963609730649 |
| 002 | 0.50000000000000 | 0.18232780382802 |         1.16591656728344 |
| 003 | 0.63636363636364 | 0.04596416746438 |         0.72174623555971 |
| 004 | 0.69005235602094 | 0.00772455219292 |         1.12751982884733 |
| 005 | 0.68202041964819 | 0.00030738417983 |         0.80384864065885 |
| 006 | 0.68232578140989 | 0.00000202241813 |         0.97481374069426 |
| 007 | 0.68232780435903 | 0.00000000053101 |         0.86774886684321 |
| 008 | 0.68232780382802 | 0.00000000000000 |         1.01567939794005 |
| 009 | 0.68232780382802 | 0.00000000000000 | 207243987.96379616856575 |


### generalizations of secant method

note: vanilla secant is a progression of points and not a bracketing method.

#### <b>regula falsi</b>


aka "method of false position". regula falsi is similar to bisection but midpoint replaced by secant-like approximation. ie, given bracketing interval $[a,b]$,

$$c = a - \tfrac{f(a)\cdot (a-b)}{f(a)-f(b)} = \tfrac{b\cdot f(a)-a\cdot f(b)}{f(a)-f(b)},$$

where $c \in [a,b]$ and next subinterval chosen to bracket root.

###### algorithm <b>regula falsi</b>

```
# given [a,b] st f(a)⋅f(b) < 0

for i = 1,2,3,...
  c = [b⋅f(a) - a⋅f(b)] / [f(a) - f(b)]
  if f(c) == 0 stop
  if f(a)⋅f(c) < 0
    b = c
  else
    a = c
  end
next
```

##### example 17

apply regula falsi on interal $[-1,1]$ to find root $r=0$ of $f(x) = x^3 - 2\,x^2 + \tfrac{3}{2}\,x$.

\begin{align}
  x_0 &= -1,\, x_1 = 1 \\
  \\
  x_2 &= \frac{x_1\cdot f(x_0) - x_0\cdot f(x_1)}{f(x_0) - f(x_1)} = \frac{1(-\tfrac{9}{2}) - (-1)(\tfrac{1}{2})}{-\tfrac{9}{2} - \tfrac{1}{2}} = \frac{4}{5}. \\
\end{align}

$f(-1)\cdot f(\tfrac{4}{5}) < 0 \implies [x_0,x_2] = [-1,\tfrac{4}{5}] \sim$ better than $\tfrac{1}{2}$ of bisection; however, sometimes its not your birthday.

###### code, example 17, regula falsi

In [ ]:
eg_17(show="data")

regula falsi: $x^3 + 2x^2 - 1.5x$, $x \in[-1.0,1.0]$

|   i |           a |        f(a) |          b |       f(b) |          c |       f(c) |   ± |
|-----|-------------|-------------|------------|------------|------------|------------|-----|
| 000 | -1.00000000 | -4.50000000 | 1.00000000 | 0.50000000 | 0.80000000 | 0.43200000 | 001 |
| 001 | -1.00000000 | -4.50000000 | 0.80000000 | 0.43200000 | 0.64233577 | 0.40333785 | 001 |
| 002 | -1.00000000 | -4.50000000 | 0.64233577 | 0.40333785 | 0.50724082 | 0.37678437 | 001 |
| 003 | -1.00000000 | -4.50000000 | 0.50724082 | 0.37678437 | 0.39079015 | 0.34043162 | 001 |


In [ ]:
ani = eg_17()
ani

###### code, example 17, secant

In [ ]:
eg_17a(show="data") # example 17, secant method

secant: $x^3 + 2x^2 - 1.5x$, $x \in[-1,1]$

expected rate of convergence: [1.19458315]

|   i |              x[i] |             e[i] |       "/e[i-1]^α |
|-----|-------------------|------------------|------------------|
| 000 | -1.00000000000000 | 1.00000000000000 |                  |
| 001 |  1.00000000000000 | 1.00000000000000 | 1.00000000000000 |
| 002 |  0.80000000000000 | 0.80000000000000 | 0.80000000000000 |
| 003 | -0.47058823529412 | 0.47058823529412 | 0.67521916496024 |
| 004 |  0.47424724729948 | 0.47424724729948 | 1.60576763820552 |
| 005 |  0.25965464146786 | 0.25965464146786 | 0.86822303833010 |
|-----|-------------------|------------------|------------------|
| 008 |  0.03994345289533 | 0.03994345289533 | 1.49983922882677 |
| 009 | -0.00643218295752 | 0.00643218295752 | 1.17833000297553 |
| 010 |  0.00035223951983 | 0.00035223951983 | 1.23876434915084 |
| 011 |  0.00000300563144 | 0.00000300563144 | 1.16216844505921 |
| 012 | -0.00000000141202 | 0.00000000141202 | 1.21542

In [ ]:
ani = eg_17a(show="ani") # example 17, secant method
ani

####<b>mullers method</b>

draw parabola $y= p(x)$ through three previous points (vs line through two previous points) and its intersection with $x$-axis closest to $x_i$ is next iteration $x_{i+1}$.

- for multiple intersections, select the one closest to previous iteration;
- if parabola misses $x$-axis then it gets complex and costs extra tuition. 👀

oscar velize [@youtube](https://www.youtube.com/watch?v=XIIEjwtkONc)

###### code, mullers method

In [ ]:
ani = eg_17c()
ani

In [ ]:
eg_17c(show="data")

step 0, parabola: -1.0 + 3.0·x - 1.0·x²
           roots: [2.6180339887498976,0.38196601125010493]


step 1, parabola: -1.0 - 6.23606798·x + 3.61803399·x²
           roots: [1.871307386268059,-0.14770058851807896]


step 2, parabola: 8.79829268 - 14.87782909·x + 5.48934138·x²
           roots: [1.8385321777112726,0.8717800572671759]


step 3, parabola: 8.00723819 - 14.15294492·x + 5.32787355·x²
           roots: [1.8392902102200412,0.8171063380988388]


step 4, parabola: 5.32800227 - 11.26393044·x + 4.54912977·x²
           roots: [1.8392867552294225,0.6367759193327744]



modified example 17 w mullers: $x^3 - x^2 - x - 1$

|   i |           a |          b |          c |         gap |       e[i] |   "/e[i-1] |
|-----|-------------|------------|------------|-------------|------------|------------|
| 000 | -2.00000000 | 0.00000000 | 2.00000000 |  2.00000000 | 0.16071324 |            |
| 001 |  0.00000000 | 2.00000000 | 2.61803399 |  0.61803399 | 0.77874723 | 4.84556973 |
| 002 |  2.00000

#### <b>inverse quadratic interpolation (IQI)</b>

similar to mullers but with parabola $x=p(y)$, which is handy for limiting the $x$-axis intesection to a single point.

consider second-degree polynomial $x = P(y)$ through points $(a,A),(b,B),(c,C)$.

\begin{align}
  P(y) &= a\,\frac{(y-B)(y-C)}{(A-B)(A-C)} + b\,\frac{(y-A)(y-C)}{(B-A)(B-C)} + c\,\frac{(y-A)(y-B)}{(C-A)(C-B)} \\
  \\
  &\Downarrow \quad P(A) = a, P(B) = b, P(C) = c, y = 0 \\
  \\
  P(0) &= c - \frac{r(r-q)(c-b) + (1-r)s(c-a)}{(q-1)(r-1)(s-1)}, \quad q = \tfrac{f(a)}{f(b)}, r = \tfrac{f(c)}{f(b)}, s = \tfrac{f(c)}{f(a)} \\
  \\
  &\Downarrow \quad a = x_i, b = x_{i+1}, c = x_{i+2}, A = f(x_i), B = f(x_{i+1}), C = f(x_{i+2}) \\
  \\
  x_{i+3} &= x_{i+2} - \frac{r(r-q)(x_{i+2}-x_{i+1}) + (1-r)s(x_{i+2}-x_i)}{(q-1)(r-1)(s-1)}, \quad q = \tfrac{f(x_i)}{f(x_{i+1})}, r = \tfrac{f(x_{i+2})}{f(x_{i+1})}, s = \tfrac{f(x_{i+2})}{f(x_i)}. \\
\end{align}

here $x_{i+3}$ replaces $x_i$ but an alternative implementation replaces the largest source of backward error.

lemonfully [@youtube](https://www.youtube.com/watch?v=3D1Uit7-SDg) [@wiki](https://en.wikipedia.org/wiki/Inverse_quadratic_interpolation)


- lemonfully points out that while this method is asymptotically faster than secants, it only is if initial points chosen well.
- oscar veliz (in the lead up to [brents method](https://www.youtube.com/watch?v=-bLSRiokgFk)) also points out this unreliability.

oh, why not. (a few reasons come to mind.)

In [ ]:
eg_17d(show="data") # oh, why not


modified example 17 w iqi: $x^3 - x^2 - x - 1$ and root [1.83928676]

|   i |          a |          b |          c |   bwe (Δy) |       e[i] |   "/e[i-1] |
|-----|------------|------------|------------|------------|------------|------------|
| 000 | 1.00000000 | 1.50000000 | 2.00000000 | 0.50000000 | 0.83928676 |            |
| 001 | 2.05964912 | 1.50000000 | 2.00000000 | 0.43554618 | 0.22036237 | 0.26255909 |
| 002 | 1.82546813 | 1.50000000 | 2.00000000 | 1.07473271 | 0.01381863 | 0.06270865 |
| 003 | 1.82546813 | 1.84037070 | 2.00000000 | 0.08066758 | 0.00108394 | 0.07844079 |
| 004 | 1.82546813 | 1.84037070 | 1.83928426 | 0.00594852 | 0.00000250 | 0.00230327 |
| 005 | 1.83928676 | 1.84037070 | 1.83928426 | 0.00001366 | 0.00000000 | 0.00001790 |
| 006 | 1.83928676 | 1.83928676 | 1.83928426 | 0.00000000 | 0.00000000 | 0.00000497 |


In [ ]:
ani = eg_17d() # a few reasons come to mind
ani

In [ ]:
# briefly

if __name__ == "__main__":

  f = lambda x: pow(x,3) + x - 1

  if True:
    x = iqi(f,[0.,0.5,1.])
    print(f"iqi, std: {x}\n")

    x = iqi(f,[0.,0.5,1.],doyourworst=True)
    print(f"iqi, bwe: {x}\n")


iqi, std: 0.6823278038280194

iqi, bwe: 0.6823278038280194



## 2 brents method


this hybrid method uses concepts of secant method, its generalizations and bisection. it expands dekkers method which uses secant backed up by bisection.

for continuous function $f$ over bounded interval $[a,b]$ where $f(a)\cdot f(b)<0$, brents method keeps track of current $x_i$ that is best in sense of backward error and bracket $[a_i,b_i]$ of root. roughly speaking brents uses IQI to replace one of $x_i,a_i,b_i$ if (1) the backward error improves and (2) the bracketing interval is cut at least in half. if that fails, the secant method is attempted. if that fails, bisection occurs which guarantees that uncertainty is lat least halved.


In [ ]:
f = lambda x: pow(x,3) - pow(x,2) - x - 1
ab = (0,2)

root_dkr,iterations = zeroin(f,ab[0],ab[1],display=0)
print(f"  zeroin: {root_dkr}, {iterations} iterations\n")

soln_sys = sp.optimize.root(f,sum(ab)/len(ab))
print(f"{soln_sys}\n")

soln_sys = sp.optimize.root(f,root_dkr)
print(f"{soln_sys}\n")


  zeroin: 1.839286755214161, 51 iterations

 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [-2.000e+00]
       x: [ 1.003e+00]
  method: hybr
    nfev: 13
    fjac: [[-1.000e+00]]
       r: [-4.710e-03]
     qtf: [ 2.000e+00]

 message: The solution converged.
 success: True
  status: 1
     fun: [ 2.220e-16]
       x: [ 1.839e+00]
  method: hybr
    nfev: 3
    fjac: [[-1.000e+00]]
       r: [-5.470e+00]
     qtf: [ 1.332e-15]



In [ ]:
# brent at scipy

f = lambda x: pow(x,3) - pow(x,2) - x - 1
ab = (0,2)

soln = sp.optimize.root_scalar(f,x0=sum(ab)/len(ab))
print(f"{soln}\n")

root,soln = sp.optimize.brentq(f,ab[0],ab[1],full_output=True)
print(f"{soln}\n")

root,soln = sp.optimize.brenth(f,ab[0],ab[1],full_output=True)
print(f"{soln}\n")


      converged: True
           flag: converged
 function_calls: 100
     iterations: 50
           root: 1.8392867552141612
         method: newton

      converged: True
           flag: converged
 function_calls: 10
     iterations: 9
           root: 1.8392867552141612
         method: brentq

      converged: True
           flag: converged
 function_calls: 10
     iterations: 9
           root: 1.8392867552141612
         method: brenth



- oscar veliz [@youtube](https://www.youtube.com/watch?v=-bLSRiokgFk) (multiple methods including brents method)

- [brents method](https://en.wikipedia.org/wiki/Brent%27s_method)
- [richard brent](https://en.wikipedia.org/wiki/Richard_P._Brent)
- [theodorus dekker](https://en.wikipedia.org/wiki/Theodorus_Dekker) with a [bonus](https://en.wikipedia.org/wiki/Dekker%27s_algorithm)

- library function documentation [@scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html) [@matlab](https://www.mathworks.com/help/matlab/ref/fzero.html)
- matlab [@online](https://matlab.mathworks.com)

## resources

meh. youre good.